In [1]:
import yaml
import os
import logging
import plotly
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import glog as log
from datetime import date

import evaluation.tools as evt
from evaluation.evaluation_lib import aggregate_all_results

In [2]:
# Parse experiment yaml file
experiments_path="../experiments/full_euroc.yaml"

# Get experiment information from yaml file.
experiment_params = yaml.load(open(experiments_path))

# Get directory where all results are stored
results_dir = os.path.expandvars(experiment_params['results_dir'])

# Collect results
stats = aggregate_all_results(results_dir)

I1219 18:48:51.022127 27499 evaluation_lib.py:44] Aggregate dataset results.


In [5]:
evt.draw_ape_boxplots_plotly(stats, False)

I1219 18:49:11.263324 27499 plotter.py:271] Creating dataframe stats.
